In [1]:
from dotenv import load_dotenv
import os
print(load_dotenv('../.env'))
print(os.environ['LANGSMITH_PROJECT'])
os.environ['LANGSMITH_TRACING']="true"
os.environ['USER_AGENT'] = 'myagent'


MODEL_NAME = "gpt-4.1"
MODEL_NAME2 = "gpt-4.1-mini"
dataset_name = "test-dataset" # change it based on the dataset name given

True
agentic-ops


## RAG APP

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def preprocess_dataset(docs_list):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=700,
        chunk_overlap=50,
        disallowed_special=()
    )
    doc_splits = text_splitter.split_documents(docs_list)
    return doc_splits
# https://huggingface.co/datasets/m-ric/transformers_documentation_en
transformers_doc = HuggingFaceDatasetLoader("m-ric/transformers_documentation_en", "text")
docs = preprocess_dataset(transformers_doc.load()[:50])

vectorstore = QdrantVectorStore.from_documents(
    docs,
    OpenAIEmbeddings(model=os.environ["EMBEDDING_MODEL"]),
    location=":memory:",
    collection_name="documentations",
)
retriever = vectorstore.as_retriever()

In [3]:
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

In [4]:
from langsmith import traceable
from openai import OpenAI
from typing import List

openai_client = OpenAI()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents, model_name):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages, model_name)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict], model_name) -> str:
    return openai_client.chat.completions.create(
        model=model_name,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str, model_name):
    documents = retrieve_documents(question)
    response = generate_response(question, documents, model_name)
    return response.choices[0].message.content

# Experiment

In [8]:
from langsmith import evaluate, Client

client = Client()

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"], MODEL_NAME)

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix=MODEL_NAME
)

View the evaluation results for experiment: 'gpt-4.1-2a180505' at:
https://smith.langchain.com/o/140fee06-18bc-5f8b-b369-ecfa45c28bd3/datasets/c238d5fb-3c75-4f2d-a84a-e8c667dddd25/compare?selectedSessions=1a6808de-d9a9-4ac0-a470-3cf19fdfc34f




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is Hugging accelerator and how do I use it?,Hugging Face Accelerate is a library that simp...,None,Hugging Face's Accelerate is a library designe...,1,3.116898,3e51eead-3f74-4b50-836d-2c9f270b8541,c9d789e6-12b7-43b8-bb78-1c8911d6916e
1,How do I fine-tune a pre-trained model from Hu...,To fine-tune a pre-trained model from Hugging ...,None,To fine-tune a pre-trained model from Hugging ...,1,3.281649,10130a73-a232-4b34-b370-d37e6ddca4b6,80d069b1-2184-4437-8837-de504da11fe3
2,How do I use PEFT for efficient fine-tuning?,To use PEFT (Parameter-Efficient Fine-Tuning) ...,None,To use PEFT (Parameter-Efficient Fine-Tuning) ...,1,4.252463,3bf8162d-0619-41ee-b439-9347b8bf2885,80c0ade3-caf7-41ff-b67f-bed23348eee9
3,How do I create a Gradio demo for my Hugging F...,To create a Gradio demo for your Hugging Face ...,None,To create a Gradio demo for your Hugging Face ...,1,2.783739,8720b535-614d-4b4f-9b76-1334aa1e5ec8,dc78b637-1d45-4013-9eae-c92a5347e2d7
4,What is a model card and how do I create one?,A model card is documentation (typically in a ...,None,A model card is a documentation file providing...,1,2.647623,8fc42409-e3bb-4a30-bc7f-2d2ed748e465,67e1c786-bc82-413b-b1d4-13d9492a3a57
5,How do I create a custom dataset with the Data...,To create a custom dataset with the Hugging Fa...,None,To create a custom dataset with the Hugging Fa...,1,2.509607,9da67a80-98c3-47ac-adb0-f7066b5fa192,ea4c28bd-3998-428a-90d3-ba6b2c2ad917
6,How do I quantize a model with bitsandbytes in...,To quantize a model with bitsandbytes in Huggi...,None,To quantize a model with bitsandbytes in Huggi...,1,2.567101,beeb1981-f60c-44d3-ad66-e880852f0b59,c9510cc2-87a0-4f2c-a529-660c5da0873c
7,How do I enable mixed precision training with ...,To enable mixed precision training with Huggin...,None,To enable mixed precision training with Huggin...,1,2.880513,bf8f4d54-4e79-410c-bbdb-a9dbf53161bc,df6fc50e-422d-4f7d-858f-c385c6300c5a
8,What is the Hugging Face Hub?,The Hugging Face Hub is an online platform and...,None,The Hugging Face Hub is a platform for sharing...,1,2.324228,c8c05254-8878-4af1-94fe-677a4c023670,283bbaf8-e2cb-4193-b4e1-c1dbb43e96a4
9,How do I use Accelerate for distributed traini...,To use Hugging Face Accelerate for distributed...,None,To use Accelerate for distributed training wit...,1,3.339923,df1aeb36-777c-4028-a809-0f9cb6ed50ee,78657d92-b96f-496e-be8f-e459cf2a4e70


## Modifying app

In [9]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"], MODEL_NAME2) 

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix=MODEL_NAME2
)

View the evaluation results for experiment: 'gpt-4.1-mini-a2211fde' at:
https://smith.langchain.com/o/140fee06-18bc-5f8b-b369-ecfa45c28bd3/datasets/c238d5fb-3c75-4f2d-a84a-e8c667dddd25/compare?selectedSessions=19f714c8-28c3-4c30-bfa8-931df80ccf7f




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is Hugging accelerator and how do I use it?,Hugging Face Accelerate is a library that simp...,None,Hugging Face's Accelerate is a library designe...,1,2.491895,3e51eead-3f74-4b50-836d-2c9f270b8541,89adc29a-73d0-4da5-85cc-e7af49f94b16
1,How do I fine-tune a pre-trained model from Hu...,"To fine-tune a pre-trained Hugging Face model,...",None,To fine-tune a pre-trained model from Hugging ...,1,2.486887,10130a73-a232-4b34-b370-d37e6ddca4b6,b8b01376-8fca-4c79-83ad-2e369cab3b09
2,How do I use PEFT for efficient fine-tuning?,The provided context does not contain specific...,None,To use PEFT (Parameter-Efficient Fine-Tuning) ...,1,2.068518,3bf8162d-0619-41ee-b439-9347b8bf2885,78793d43-fbb0-450e-8764-e982d2b6ef67
3,How do I create a Gradio demo for my Hugging F...,The provided context does not include specific...,None,To create a Gradio demo for your Hugging Face ...,1,2.271339,8720b535-614d-4b4f-9b76-1334aa1e5ec8,9ce86bad-436b-47b7-8dd1-950307bd1b81
4,What is a model card and how do I create one?,A model card is a README.md file in your model...,None,A model card is a documentation file providing...,1,2.042482,8fc42409-e3bb-4a30-bc7f-2d2ed748e465,5bee5e48-2b97-47dd-a1cf-11fbaa258247
5,How do I create a custom dataset with the Data...,To create a custom dataset with the Datasets l...,None,To create a custom dataset with the Hugging Fa...,1,1.978712,9da67a80-98c3-47ac-adb0-f7066b5fa192,4c1c2701-0b3f-4650-9d79-dedb745a93e6
6,How do I quantize a model with bitsandbytes in...,To quantize a model with bitsandbytes in Huggi...,None,To quantize a model with bitsandbytes in Huggi...,1,2.633849,beeb1981-f60c-44d3-ad66-e880852f0b59,dc3d2cc4-0003-4015-9213-d1633273d73a
7,How do I enable mixed precision training with ...,To enable mixed precision training with Huggin...,None,To enable mixed precision training with Huggin...,1,2.627610,bf8f4d54-4e79-410c-bbdb-a9dbf53161bc,2b95ed31-321c-47ad-8b12-6bad96532b37
8,What is the Hugging Face Hub?,The Hugging Face Hub is a platform that hosts ...,None,The Hugging Face Hub is a platform for sharing...,1,1.966247,c8c05254-8878-4af1-94fe-677a4c023670,2e667036-f7ca-4892-a21c-6bd4fdffbc46
9,How do I use Accelerate for distributed traini...,To use Accelerate for distributed training wit...,None,To use Accelerate for distributed training wit...,1,4.904827,df1aeb36-777c-4028-a809-0f9cb6ed50ee,5cb11812-023f-4a41-841a-d2658493fb43


## DEMO UI experiments
* UI evaluator run
* this local run
* the evaluation UI